In [55]:
import re
import torch
from datasets import load_dataset
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments, pipeline, set_seed

In [56]:
dataset=load_dataset('Heralax/Augmental-Dataset')
print (dataset)

DatasetDict({
    train: Dataset({
        features: ['completion', 'history', 'scenario', 'speaker'],
        num_rows: 7831
    })
})


In [57]:
dataset=dataset.filter(lambda x: False if x['speaker']!='Kurisu' else True)
train_dataset=dataset['train']
kurisu_replicas=train_dataset['completion']
def use_regex(input_text):
    out=re.findall(r'"(.*?)"', input_text)
    if not out:
        return ""
    else:
        return out[0]
kurisu_replicas=[use_regex(inp) for inp in kurisu_replicas]
kurisu_replicas=[x for x in kurisu_replicas if x]
data=""
for item in kurisu_replicas:
    data=data+item+' '#+'<|endoftext|>'
print(data[:100])
with open('kurisu.txt', 'w') as f:
    f.write(data)

Could you come with me for a moment? Huh? Stop fooling around and come with me. What are you talking


In [58]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset


def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator


def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  token_ids = train_dataset.examples
  decoded_text = tokenizer.decode(token_ids[0])
  print(decoded_text)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)
      
  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
      
  trainer.train()
  trainer.save_model()

In [59]:
train_file_path = "kurisu.txt"
model_name = 'gpt2'
output_dir = 'gpt2-kurisu'
overwrite_output_dir = True
per_device_train_batch_size = 8
num_train_epochs = 40.0
save_steps = 500 #Lower this next training.

In [60]:
# It takes about 30 minutes to train in colab.
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/home/alex/Documents/IOAI/Notebooks/venv/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Could you come with me for a moment? Huh? Stop fooling around and come with me. What are you talking about? I just need to ask you something. What's with this 'Organization' stuff?...So you talk to yourself. What were you trying to tell me earlier? About fifteen minutes ago. Before the conference started. You were trying to tell me something, right? You looked really upset. You looked like you were going to start crying any second. Why? Have we met before? And how do you know my name? Huh? I'm sorry! Is there something wrong? Hey, that hurts!


Step,Training Loss
500,2.079500
1000,0.804000


In [61]:
generator = pipeline('text-generation', model='gpt2-kurisu', device='cuda')
generator2 = pipeline('text-generation', model='gpt2', device='cuda')

In [70]:
set_seed(42)
message="""
The
"""
max_len=200
num_seq=3
#set_seed(42)
print(generator(message, max_length=max_len, num_return_sequences=num_seq)[0]['generated_text'].replace(message, ''))
print ('----------------------------------------------')
print(generator2(message, max_length=max_len, num_return_sequences=num_seq)[0]['generated_text'].replace(message, ''))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



-- Time Leap Machine. C-come on! Time travel to the past is possible right now. It wouldn't have been possible five hours ago. It's so unbelievably dangerous! How could you plan to sneak in a time machine!? It's dangerous to even think about it, Okabe. This is Russia. Taking a break at night is practically forbidden here. Actually, I'm not even sure what to say. Not a chance. This isn't about me. Here, I want to expose the lies of the capitalist system, and expose the failures of the past five years. Shut up, perv. Remember that I said I'm not a hypocrite? You win here by being strong, Okabe. If you want your revenge, I'm almost certain you will. But I'm asking for your loyalty, okay? You're such a wimp, you know? Don't take everything I say seriously. Remember that I said I'm not a hypocrite? You
----------------------------------------------

A

Meter


As well

A

Meter

As well

I

D

V

B

D

V

B

A

Meter

As well

I

D

V

B

A

Meter

As well

I

D

V

B

A

Meter

As well

I